In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")

import warnings
import os
import json
#Suppressing all warnings
warnings.filterwarnings("ignore")

%matplotlib inline

def get_files(root):
    return [os.path.join(root, f) for f in os.listdir(root)]

In [ ]:
folders = get_files("messages/inbox/") + get_files("messages/archived_threads/")
users = []

for f in folders:
    user = f.split("/")[-1]
    if ord(user[0]) >= 65 and ord(user[0]) <= 90:
        continue
        
    if "files" in os.listdir(f):
        continue
        
    users.append(f)
    
len(users)

In [ ]:
import sys
import json
from datetime import datetime


def fixup_str(text):
    try:
        return text.encode('latin1').decode('utf8')
    except:
        return ""


def fixup_list(l):
    return [fixup(e) for e in l]


def fixup_dict(dct):
    return {fixup_str(k): fixup(v) for k, v in dct.items()}


def fixup(e):
    if isinstance(e, dict):
        return fixup_dict(e)
    if isinstance(e, list):
        return fixup_list(e)
    if isinstance(e, str):
        return fixup_str(e)
    return e


def filter_messages(messages):
    for message in messages:
        if message['type'] == 'Generic':
            yield message


def format_message(message):
#     print(message)
    sender = message['sender_name']
    timestamp = datetime.fromtimestamp(message['timestamp_ms']/1000)
    try:
        content = message['content']
    except KeyError:
        content = ""
        
    return f'{timestamp}\t{sender}\t{content}'


def main(fname):
    mess = []
    with open(fname, 'r') as f:
        data = fixup(json.loads(f.read()))
        messages = data['messages']
        participants = data["participants"]
        
    for message in filter_messages(messages):
        mess.append([len(participants), format_message(message)])
        
    return mess

# json.load(open("messages/inbox/facebookuser_q1c23rfqfq/message_1.json"), object_hook=parse_obj)
main("messages/inbox/facebookuser_q1c23rfqfq/message_1.json")

In [ ]:
df_data = []
failed = []
failed_file = []
for u in users:
    mess_files = get_files(u)    
    for mf in mess_files:
#         print(mf)
        if not mf.endswith(".json"):
            continue
        try:
            data = main(mf)
        except:
            failed_file.append(mf)
            continue
            
        for lp, d in data:
            try:
                date, name, mess = d.split("\t")
                df_data.append([date, u.split("/")[-1], lp, name, mess])
            except:
                failed.append(d)
            
len(df_data)

In [ ]:
failed_file

In [ ]:
df = pd.DataFrame(df_data, columns=["DateTime", "Name", "NumPart", "UserName", "Message"])
df.sample(10)

In [ ]:
df.sort_values(by="DateTime")

In [ ]:
direct_mess = {}
for dt, _, lp, user, _ in df.to_numpy().tolist():
    if int(lp) > 2:
        continue
    
    if user not in direct_mess:
        direct_mess[user] = 1
    else:
        direct_mess[user] += 1
direct_mess

In [ ]:
sorted_direct_mess = sorted(direct_mess.items(), key=lambda x: -x[1])
sorted_direct_mess

In [ ]:
#CountPlot
fig, ax = plt.subplots(figsize=(20, 15))
ax.set_title('Number of direct messages', fontsize=20)
sns.barplot(y=[s[0] for s in sorted_direct_mess[1:30]], x=[s[1] for s in sorted_direct_mess[1:30]], orient="h")
ax.set_xlabel('Messages', fontsize=15)
ax.set_ylabel('Friends', fontsize=15)
plt.show()

In [ ]:
interact = {}

for dt, _, lp, user, _ in df.to_numpy().tolist():
    if user not in interact:
        interact[user] = 1 / lp
    else:
        interact[user] += 1 / lp
        
sorted_interact = sorted(interact.items(), key=lambda x: -x[1])
sorted_interact

In [ ]:
#CountPlot
fig, ax = plt.subplots(figsize=(20, 15))
ax.set_title('Overall Messenger Interaction', fontsize=20)
sns.barplot(y=[s[0] for s in sorted_interact[1:30]], x=[s[1] for s in sorted_interact[1:30]], orient="h")
ax.set_xlabel('Message score (groups included)', fontsize=15)
ax.set_ylabel('Friends', fontsize=15)
plt.show()

In [ ]:
interact_date = {}

for dt, _, lp, user, _ in df.to_numpy().tolist():
    dt = int(dt.split(" ")[0].replace("-", ""))
    if user not in interact_date:
        interact_date[user] = 1 / lp * (dt - 20130225) ** 3
    else:
        interact_date[user] += 1 / lp * (dt - 20130225) ** 3
        
sorted_interact_date = sorted(interact_date.items(), key=lambda x: -x[1])
sorted_interact_date

In [ ]:
#CountPlot
fig, ax = plt.subplots(figsize=(20, 15))
ax.set_title('Overall Messenger Interaction (prioritize recency)', fontsize=20)
sns.barplot(y=[s[0] for s in sorted_interact_date[1:30]], x=[s[1] for s in sorted_interact_date[1:30]], orient="h")
ax.set_xlabel('Message score (groups included)', fontsize=15)
ax.set_ylabel('Friends', fontsize=15)
plt.show()

In [ ]:
from datetime import datetime
df["Year"] = df["DateTime"].apply(lambda x: x.split(" ")[0].split("-")[0])
hm_year = {}

def get_heatmap_by_year(year):
    df_me = df[(df["UserName"] == "Nguyễn Tài Long") & (df["Year"] == str(year))]
    df_me["Hour"] = df_me["DateTime"].apply(lambda x: x.split(" ")[1].split(":")[0])
    df_me["Day"] = df_me["DateTime"].apply(lambda x: datetime.strptime(x.split(".")[0], "%Y-%m-%d %H:%M:%S").weekday())
    tmp = df_me.groupby(["Day", "Hour"]).count().reset_index()[["Day", "Hour", "Message"]]
    hm = [[0 for _ in range(24)] for _ in range(7)]
    for d, h, m in tmp.to_numpy().tolist():
        hm[int(d)][int(h)] = m
    
    return hm

for year in range(2013, 2021):
    hm_year[year] = get_heatmap_by_year(year)

In [ ]:
fig, axes = plt.subplots(nrows=8, ncols=1, figsize=(20, 5 * 8), dpi=200)
for year in range(2013, 2021):    
    axes[year - 2013].set_title('Usage Heat Map - {}'.format(year), fontsize=20)
    sns.heatmap(data=hm_year[year], xticklabels=[i for i in range(24)], ax=axes[year - 2013], 
                yticklabels=["Mon", "Tue", 'Wed', "Thu", "Fri", "Sat", "Sun"],
               cmap=sns.light_palette("seagreen", as_cmap=True))
plt.show()
plt.savefig("heatmaps.png")

In [ ]:
df_me = df[df["UserName"] == "Nguyễn Tài Long"]

all_my_text = df["Message"].tolist()
len(all_my_text)

In [ ]:
from nltk import ngrams
import re
_CHARACTERS_DIGITS = 'abcdeghiklmnopqrstuvxyàáâãèéêìíòóôõùúýăđĩũơưạảấầẩẫậắằẳẵặẹẻẽếềểễệỉịọỏốồổỗộớờởỡợụủứừửữựỳỵỷỹ0123456789 '
most_common_phrases = {}
for text in all_my_text:
    text = re.sub("[^{}]".format(_CHARACTERS_DIGITS), " ", text.lower())
    for ph in ngrams(text.split(), 4):
        ph = " ".join(list(ph))
        if ph not in most_common_phrases:
            most_common_phrases[ph] = 1
        else:
            most_common_phrases[ph] += 1
            
    for ph in ngrams(text.split(), 3):
        ph = " ".join(list(ph))
        if ph not in most_common_phrases:
            most_common_phrases[ph] = 1
        else:
            most_common_phrases[ph] += 1
            

        
sorted_phrases = sorted(most_common_phrases.items(), key=lambda x: -x[1])
sorted_phrases

In [ ]:
!pip install wordcloud
!pip install multidict

In [ ]:
interesting_phrases = [('nói chung là', 549),
 ('nói chuyện với', 388),
 ('là cái gì', 341),
 ('liên quan đến', 341),
 ('làm gì có', 338),
 ('ý t là', 319),
 ('ở đâu đấy', 293),
 ('đi đá bóng', 286),
 ('quan trọng là', 280),
 ('lúc nào cũng', 275),
 ('t nghĩ là', 264),
 ('có gì đâu', 255),
 ('liên quan gì', 253),
 ('hay sao ý', 252),
 ('ở công ty', 238),
 ('làm thế nào', 235),
 ('có vấn đề', 231),
 ('đang làm gì', 226),
 ('ko phải là', 226),
 ('như thế nào', 225),
 ('để làm gì', 220),
 ('mối quan hệ', 218),
 ('vấn đề là', 206),
 ('đang ở đâu', 204),
 ('có đứa nào', 201),
 ('thế nào rồi', 195),
 ('cái gì đấy', 194),
 ('đi xem phim', 189),
 ('xem thế nào', 184),
 ('đi xe máy', 182),
 ('em cảm ơn', 180),
 ('ở đâu thế', 178),
 ('vấn đề gì', 178),
 ('có thể là', 178),
 ('ko có gì', 177),
 ('t cũng k', 171),
 ('tất cả các', 169),
 ('đang ở nhà', 168),
 ('là thế nào', 168),
 ('làm gì đấy', 168),
 ('không có gì', 165),
 ('quan tâm đến', 165),
 ('mấy hôm nay', 163),
 ('cũng có thể', 163),
 ('có thời gian', 162),
 ('mình có thể', 160),
 ('đi chơi với', 159),
 ('cuối tuần này', 158),
 ('làm cái gì', 155),
 ('lên công ty', 155),
 ('ở infore', 154),
 ('em có thể', 153),
 ('gì đâu mà', 153),
 ('chứ k phải', 149),
 ('có cơ hội', 147),
 ('có thể làm', 145),
 ('câu trả lời', 142),
 ('hình như là', 142),
 ('về đến nhà', 141),
 ('k đi đc', 137),
 ('thực ra là', 136),
 ('cảm ơn anh', 136),
 ('tóm lại là', 135),
 ('mừng sinh nhật', 134),
 ('đứa con gái', 134),
 ('kiểu gì cũng', 133),
 ('thôi ngủ đi', 133),
 ('cái này thì', 132),
 ('thế này thì', 132),
 ('bọn con gái', 128),
 ('có khả năng', 127),
 ('hôm nay đi', 126),
 ('anh lê công thành', 125),
 ('dạ vâng ạ', 124),
 ('chúc mừng sinh nhật', 124),
 ('cho các bạn', 124),
 ('công ty mình', 123),
 ('hạnh má mì', 123),
 ('cho em hỏi', 122),
 ('có cái gì', 121),
 ('mấy cái này', 121),
 ('t cũng thế', 121),
 ('nào cũng đc', 120),
 ('t ko biết', 120),
 ('có chuyện gì', 118),
 ('sao tự nhiên', 118),
 ('đi ngủ đây', 117),
 ('có kinh nghiệm', 117),
 ('đi du lịch', 116),
 ('ý em là', 116),
 ('chuẩn bị đi', 113),
 ('có gì hot', 113),
 ('làm gì mà', 112),
 ('xử lý ảnh', 112),
 ('mai mấy h', 110),
 ('cái gì đó', 110),
 ('bình thường mà', 109),
 ('chả có gì', 109),
 ('t đang ở', 108),
 ('thế nào là', 108),
 ('t cũng ko', 108),
 ('phải đi học', 108),
 ('đi du học', 108),
 ('làm gì đâu', 107),
 ('có mấy đứa', 107),
 ('đến đâu rồi', 107),
 ('có ai đi', 107),
 ('có gì mà', 106),
 ('cái đấy thì', 106),
 ('t đi học', 106),
 ('phụ thuộc vào', 105),
 ('nào cũng được', 105),
 ('xem bóng đá', 105),
 ('em nghĩ là', 104),
 ('tuần sau đi', 104),
 ('mới bắt đầu', 104),
 ('được không ạ', 104),
 ('có vấn đề gì', 103),
 ('chứ không phải', 103),
 ('không phải là', 103),
 ('thì t cũng', 103),
 ('ví dụ như', 102),
 ('làm gì thế', 101),
 ('cũng k biết', 101),
 ('k có gì', 101),
 ('thì nó sẽ', 101),
 ('thôi ngủ đây', 101),
 ('các anh chị', 101),
 ('thế nào ạ', 100),
 ('thế còn gì', 100),
 ('đi uống bia', 75)]


In [ ]:
import multidict as multidict
from wordcloud import WordCloud, ImageColorGenerator
from PIL import Image
from scipy.ndimage import gaussian_gradient_magnitude

fullTermsDict = multidict.MultiDict()
    
for k, v in interesting_phrases:
    fullTermsDict.add(k, v)

# load image. This has been modified in gimp to be brighter and have more saturation.
parrot_color = np.array(Image.open("parrot-by-jose-mari-gimenez2.jpg"))
# subsample by factor of 3. Very lossy but for a wordcloud we don't really care.
parrot_color = parrot_color[::3, ::3]

# create mask  white is "masked out"
parrot_mask = parrot_color.copy()
parrot_mask[parrot_mask.sum(axis=2) == 0] = 255

# some finesse: we enforce boundaries between colors so they get less washed out.
# For that we do some edge detection in the image
edges = np.mean([gaussian_gradient_magnitude(parrot_color[:, :, i] / 255., 2) for i in range(3)], axis=0)
parrot_mask[edges > .08] = 255

# wc = WordCloud(background_color="black", max_words=1000, width=3000, height=2000)
wc = WordCloud(max_words=2000, mask=parrot_mask, max_font_size=40, random_state=42, relative_scaling=0, scale=1.5)
# generate word cloud
wc.generate_from_frequencies(fullTermsDict)

fig, ax = plt.subplots(figsize=(20, 15))
ax.imshow(wc, interpolation="bilinear")
ax.axis("off")
plt.show()

# create coloring from image
image_colors = ImageColorGenerator(parrot_color)
wc.recolor(color_func=image_colors)
# plt.figure(figsize=(10, 10))
# plt.imshow(wc, interpolation="bilinear")
wc.to_file("parrot_new.png")